In [1]:
# Data  Extraction and Text Analysis - Task for Data Science Internship at Blackcoffer

In [128]:
import numpy  as np
import pandas as pd
import re
import string
import urllib.request
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import html
import requests

In [130]:
#Reading the data from the Excel file into a dataFrame and eader column names are being assigned as the dataframe column names
input_file  = pd.read_excel("ciklist.xls", header=None) 
header  = input_file.iloc[0,:]                          
input_file.columns  = header                                      
secfname_or  = input_file.copy()                             
secfname_or       = secfname_or.iloc[1:,5]                      
file  = input_file.iloc[1:,:]                         
file.head()     

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
1,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,edgar/data/3662/0000950170-98-000413.txt
2,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,edgar/data/3662/0000950170-98-001001.txt
3,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,edgar/data/3662/0000950170-98-002145.txt
5,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [131]:
# Now adding https://www.sec.gov/Archives/ to every cells of column F (ciklist.xls) to access link to the financial report as mentioned in the objective
pd.options.mode.chained_assignment = None  # default='warn'

#Add the below link at the prefix of all the values in SECFNAME column
link = 'https://www.sec.gov/Archives/'
file['SECFNAME'] = link + file['SECFNAME'].astype(str)

#Now all the values in the SECFNAME column have become hyperlinks linking to 
#HTML pages
file.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
1,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
2,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
5,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...


In [117]:
output_file =  pd.read_excel("Output Data Structure.xls", header=None) 
Header = output_file.iloc[0,:] 
output_file.columns  = Header
outfile  = output_file.iloc[1:,:] 
outfile.columns
#extracting columns to merge with input file 

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'SECFNAME',
       'mda_positive_score', 'mda_negative_score', 'mda_polarity_score',
       'mda_average_sentence_length', 'mda_percentage_of_complex_words',
       'mda_fog_index', 'mda_complex_word_count', 'mda_word_count',
       'mda_uncertainty_score', 'mda_constraining_score',
       'mda_positive_word_proportion', 'mda_negative_word_proportion',
       'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion',
       'qqdmr_positive_score', 'qqdmr_negative_score', 'qqdmr_polarity_score',
       'qqdmr_average_sentence_length', 'qqdmr_percentage_of_complex_words',
       'qqdmr_fog_index', 'qqdmr_complex_word_count', 'qqdmr_word_count',
       'qqdmr_uncertainty_score', 'qqdmr_constraining_score',
       'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
       'qqdmr_uncertainty_word_proportion',
       'qqdmr_constraining_word_proportion', 'rf_positive_score',
       'rf_negative_score', 'rf_polarity

In [118]:
#now connecting the required output data structure new coumns to file
new_columns = ['mda_positive_score', 'mda_negative_score', 'mda_polarity_score', 'mda_average_sentence_length', 'mda_percentage_of_complex_words', 'mda_fog_index',
               'mda_complex_word_count', 'mda_word_count', 'mda_uncertainty_score', 'mda_constraining_score', 'mda_positive_word_proportion', 'mda_negative_word_proportion',
               'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion','qqdmr_positive_score','qqdmr_negative_score',
               'qqdmr_polarity_score','qqdmr_average_sentence_length','qqdmr_percentage_of_complex_words', 'qqdmr_fog_index','qqdmr_complex_word_count',
               'qqdmr_word_count', 'qqdmr_uncertainty_score', 'qqdmr_constraining_score', 'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
               'qqdmr_uncertainty_word_proportion', 'qqdmr_constraining_word_proportion', 'rf_positive_score', 'rf_negative_score',
               'rf_polarity_score', 'rf_average_sentence_length', 'rf_percentage_of_complex_words', 'rf_fog_index', 'rf_complex_word_count',
               'rf_word_count', 'rf_uncertainty_score', 'rf_constraining_score', 'rf_positive_word_proportion', 'rf_negative_word_proportion',
               'rf_uncertainty_word_proportion', 'rf_constraining_word_proportion', 'constraining_words_whole_report']

file = pd.concat([file,pd.DataFrame(columns=new_columns)]) 
file = file.reset_index()                                  
del file['index']                                              
file.head()                                                   

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
#Fetching the Stop words list
with open('StopWords_GenericLong.txt') as f:
    Stopwords = list(f) 
    Stopwords = [i.rstrip() for i in Stopwords]
    Stopwords = [j.lower() for j in Stopwords]
    
#Fetching the list of Positive and Negative words
Positive_words = pd.read_excel("LoughranMcDonald_SentimentWordLists_2018.xls", "Positive", header=None)
Negative_words = pd.read_excel("LoughranMcDonald_SentimentWordLists_2018.xls", "Negative", header=None)

#Seperating all the positive and Negative words into two separate lists
Positive_words = Positive_words[0]                           
Negative_words = Negative_words[0]

#Keeping all the positive and Negative words which are not present in stop words list
Positive_cleaned = list(set(Positive_words)-set(Stopwords)) 
Negative_cleaned = list(set(Negative_words)-set(Stopwords)) 

#Converting the Positive and Negative words to Lowercase
Positive_cleaned = [i.lower() for i in Positive_cleaned]
Negative_cleaned = [i.lower() for i in Negative_cleaned]

#Fetching the uncertainity and constarining words from the Excel sheet
Uncertainity_dict = pd.read_excel("uncertainty_dictionary.xls", header=None)
Constraining_dict = pd.read_excel("constraining_dictionary.xls", header=None)

#Passing all the above obtained words into separate lists and converting it to Lowercase
Uncertainity_words = Uncertainity_dict[0].tolist()
Uncertainity_words = [i.lower() for i in Uncertainity_words]

Constraining_words = Constraining_dict[0].tolist()
Constraining_words = [i.lower() for i in Constraining_words]

In [119]:
#function for calculating the average length of the sentence
def avg_sentence_len(paragraph):
    #Calculating the number of sentences in the whole data
    no_sentence = len(sent_tokenize(paragraph))                               
    tokens = nltk.word_tokenize(paragraph)                               
    no_words = len(tokens)                                        
    #Calculating the Average Sentence length
    if (no_sentence != 0):
        avg_sentence_len = round(no_words/no_sentence)                     
    else:
        avg_sentence_len = 0
    return avg_sentence_len

In [120]:
def data_cleaning(data1, data2, data3):
    # here we get all the dat collected from the site, then tokenize the list and conbine  all the cleaned data into string of words
    if (data1 != 0):
        data1 = nltk.word_tokenize(data1)                                     
        data1 = [word for word in data1 if word.lower() not in Stopwords]  
        data1 = ' '.join(data1)                                                                                    
    if (data2 != 0):
        data2 = nltk.word_tokenize(data2)                                   
        data2 = [word for word in data2 if word.lower() not in Stopwords]  
        data2 = ' '.join(data2)                                                                      
    if (data3 != 0):
        data3 = nltk.word_tokenize(data3)                                   
        data3 = [word for word in data3 if word.lower() not in Stopwords]  
        data3 = ' '.join(data3)                                             

    return data1, data2, data3

In [121]:
def fetch_sections_data(data):
    # "MANAGEMENT'S DISCUSSION AND ANALYSIS" section     
    topic1 = "MANAGEMENT'S DISCUSSION AND ANALYSIS"
    topic1_start = [m.start() for m in re.finditer(topic1, data)]

    if (len(topic1_start) == 0):
        topic1 = "MANAGEMENTS DISCUSSION AND ANALYSIS"
        topic1_start = [m.start() for m in re.finditer(topic1, data)]

    if (len(topic1_start) != 0):
    #topic_1_data has the data from the point where topic1 begins
        topic_1_data = data[topic1_start[0]:]
        item = "ITEM"
        topic1_end = [m.start() for m in re.finditer(item, topic_1_data)]
        if (len(topic1_end) == 0):
            topic_1_data = topic_1_data[0:]
        else:  
            topic_1_data = topic_1_data[0:topic1_end[0]-1]
    #topic_1_data ends at the point where topic1 ends
    if (len(topic1_start) == 0):
        topic_1_data = 0  
     
     #SECTION 2 - Quantitative and Qualitative Disclosures 
    topic2 = "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK"
    topic2_start = [m.start() for m in re.finditer(topic2, data)]
  
    if (len(topic2_start) != 0):
        topic_2_data = data[topic2_start[0]:]
        item = "ITEM"
        topic2_end = [m.start() for m in re.finditer(item, topic_2_data)]
        if (len(topic2_end) == 0):
            topic_2_data = topic_2_data[0:]
        else:  
            topic_2_data = topic_2_data[0:topic2_end[0]-1]     
  
    if (len(topic2_start) == 0):
        topic_2_data = 0 

  #SECTION 3 - Risk Factors
    topic3 = "RISK FACTORS"
    topic3_start = [m.start() for m in re.finditer(topic3, data)]

    if (len(topic3_start) != 0):
        topic_3_data = data[topic3_start[0]:]
        item = "ITEM"
        topic3_end = [m.start() for m in re.finditer(item, topic_3_data)]
        if (len(topic3_end) == 0):
            topic_3_data = topic_3_data[0:]
        else:  
            topic_3_data = topic_3_data[0:topic3_end[0]-1]    

    if (len(topic3_start) == 0):
        topic_3_data = 0         

    return topic_1_data, topic_2_data, topic_3_data

In [122]:
def constr_whole_report(x):
    #Removing the Punctuations from the string before tokenizing & tokenizing the string into a list of words
    x      = x.translate(str.maketrans('','',string.punctuation)) 
    tokens = nltk.word_tokenize(x)                                
    tokens = [x.lower() for x in tokens]                          

    constr_whole_words = 0
    for word in tokens:                                         

        if word in Constraining_words:                                              
            constr_whole_words += 1     
      
    return constr_whole_words

In [123]:
#functrion for counting the word count and complex word count
def complex_word_count(word, complex_count):
    word   = word.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations
    tokens = nltk.word_tokenize(word)                                
    no_words = len(tokens)                                           
    complex_count = 0                                                
    for word in tokens:
        word = word.lower()                                            
        vowels = "aeiou"    
        if (word.endswith(("es", "ed"))):  #We are ignoring the words ending with es or ed in our calculation
            count = 0
        else:
            count = 0
        for c in word:
            if (c in vowels):
                count = count + 1   
    #Counting the number of Vowels in each word,if number of vowels > 2 then we are incrementing complex_count variable by 1
        if (count > 2):                                             
            complex_count = complex_count + 1 
    percentage_complex_words = complex_count/no_words                      
    return no_words, complex_count, percentage_complex_words

In [124]:
#FUNCTION FOR CALCULATING POSITIVE SCORE, NEGATIVE SCORE AND POLARITY SCORE   
def positive_negative_pololarity_score(data):

    data   = data.translate(str.maketrans('','',string.punctuation)) 
    tokens = nltk.word_tokenize(data)                                #Tokenizing the string into a list of words
    tokens = [data.lower() for data in tokens]                          
    positive_words = negative_words = 0

    for word in tokens:
        if word in Positive_cleaned:
            positive_words += 1
        if word in Negative_cleaned:
            negative_words -= 1 

    positive_len = positive_words
    negative_len = negative_words * -1
    polarity_score = (positive_len - negative_len)/((positive_len + negative_len) + 0.000001)
    return positive_len, negative_len, polarity_score

In [125]:
#function for calculating uncertainty and constraining score 
#If word is present in uncertainity word list we increment uncertainty_words
#If word is present in constaining word list we increment constraining_words
def uncertainty_constraining_score(data):
    data   = data.translate(str.maketrans('','',string.punctuation)) 
    tokens = nltk.word_tokenize(data)                                
    tokens = [data.lower() for x in tokens]                          
    uncertainty_words = constraining_words = 0
    for word in tokens:                                         
        if word in Uncertainity_words:                              
            uncertainty_words  += 1
        if word in Constraining_words:                                             
            constraining_words += 1 
    uncertainty_count  = uncertainty_words
    constraining_count = constraining_words
    return uncertainty_count, constraining_count

In [126]:
## we here apply loop through the column od file. In every iteration we fetch url data from the SECFNAME column in the file dataframe.
for i in range(len(file)):                    
  
    url = file.iloc[i,5]                        
    data = urllib.request.urlopen(url).readlines()  #Reading the data from the URL link page
  
   # Decoding the variable to remove bytes object at he begining and removing blank lines and new line character
    data = [str(s, 'utf-8') for s in data]          
    data = [s.strip('\n') for s in data]            
    data = list(filter(None, data))                 
  #After doing the above operations the number of iterations required now would be less

  #Converting the list of strings into a single string to increase speed of operation instead of looping again
    data = " ".join(data)

  #Removing the Tables from the data
    data = re.sub("(?is)<table[^>]*>(.*?)<\/table>", "", data)

  #REGEX operations to remove the HTML content from our data and get a clean text data
    html_regex = re.compile(r'<.*?>')
    data = re.sub(html_regex,'',data)
    data = data.replace('&nbsp;','')
    input_data = re.sub(r'&#\d+;', '', data)   

    constr_whole_count = constr_whole_report(input_data)
    file.loc[i, 'constraining_words_whole_report'] = constr_whole_count

  #Fetching the sections data using the fetch_sections_data()
    topic1_data, topic2_data, topic3_data = fetch_sections_data(input_data)

  #Removing the Stop words, punctuations in our data
    topic1_data, topic2_data, topic3_data = data_cleaning(topic1_data, topic2_data, topic3_data)


# values for "MANAGEMENT'S DISCUSSION AND ANALYSIS" section   |
    if (topic1_data != 0):
    #Calculating the positive score, negative score and Polarity score
        pos_score1, neg_score1, pol_score1 = positive_negative_pololarity_score(topic1_data)

        file.loc[i, 'mda_positive_score'] = pos_score1
        file.loc[i, 'mda_negative_score'] = neg_score1 
        file.loc[i, 'mda_polarity_score'] = pol_score1

    #Calculating the average sentence length for our section of data
        avg_sent_len_val1 = avg_sentence_len(topic1_data)
        file.loc[i, 'mda_average_sentence_length'] = avg_sent_len_val1

    #Calculating the word count, complex word count and percentage of complex words
        complex_count = 0
        word_count1, complex_count1, perc_complex_words1 = complex_word_count(topic1_data, complex_count)
        file.loc[i, 'mda_word_count']  = word_count1
        file.loc[i, 'mda_complex_word_count'] = complex_count1
        file.loc[i, 'mda_percentage_of_complex_words'] = perc_complex_words1

    #Calculating the Fog index value 
        fog_index1 = 0.4 * (avg_sent_len_val1 + perc_complex_words1)
        file.loc[i, 'mda_fog_index'] = fog_index1

    #Calculating the uncertainity score and constraining score
        uncer_score1, constr_score1 = uncertainty_constraining_score(topic1_data)
        perc_uncer1  = uncer_score1/word_count1
        perc_constr1 = constr_score1/word_count1
        file.loc[i, 'mda_uncertainty_score']  = uncer_score1
        file.loc[i, 'mda_constraining_score']  = constr_score1

    #Calculating the Uncertainity and Constraining word proportion
        file.loc[i, 'mda_uncertainty_word_proportion']  = perc_uncer1
        file.loc[i, 'mda_constraining_word_proportion'] = perc_constr1
    
    #Calculating the Positive and Negative word proportion
        perc_positive1 = pos_score1/word_count1
        perc_negative1 = neg_score1/word_count1
        file.loc[i, 'mda_positive_word_proportion']  = perc_positive1
        file.loc[i, 'mda_negative_word_proportion']  = perc_negative1

    else:
    #If the section 1 data is not available, all the below values should be 0
        file.iloc[i, 6:21] = 0

#  Calculating the values for QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK   
    if (topic2_data != 0):  
    
    #Calculating the positive score, negative score and Polarity score
        pos_score2, neg_score2, pol_score2 = positive_negative_pololarity_score(topic2_data)
    
        file.loc[i, 'qqdmr_positive_score'] = pos_score2
        file.loc[i, 'qqdmr_negative_score'] = neg_score2 
        file.loc[i, 'qqdmr_polarity_score'] = pol_score2
    #Calculating the average sentence length for our section of data
        avg_sent_len_val2 = avg_sentence_len(topic2_data)
        file.loc[i, 'qqdmr_average_sentence_length'] = avg_sent_len_val2
    #Calculating the word count, complex word count and percentage of complex words
        complex_count = 0
        word_count2, complex_count2, perc_complex_words2 = complex_word_count(topic2_data, complex_count)
        file.loc[i, 'qqdmr_word_count']  = word_count2
        file.loc[i, 'qqdmr_complex_word_count']  = complex_count2
        file.loc[i, 'qqdmr_percentage_of_complex_words'] = perc_complex_words2

    #Calculating the Fog index value 
        fog_index2 = 0.4 * (avg_sent_len_val2 + perc_complex_words2)
        file.loc[i, 'qqdmr_fog_index'] = fog_index2
    #Calculating the uncertainity score and constraining score
        uncer_score2, constr_score2 = uncertainty_constraining_score(topic2_data)
        perc_uncer2  = uncer_score2/word_count2
        perc_constr2 = constr_score2/word_count2
        file.loc[i, 'qqdmr_uncertainty_score']  = uncer_score2
        file.loc[i, 'qqdmr_constraining_score']  = constr_score2
    #Calculating the Uncertainity and Constraining word proportion
        file.loc[i, 'qqdmr_uncertainty_word_proportion']  = perc_uncer2
        file.loc[i, 'qqdmr_constraining_word_proportion'] = perc_constr2
    #Calculating the Positive and Negative word proportion
        perc_positive2 = pos_score2/word_count2
        perc_negative2 = neg_score2/word_count2
        file.loc[i, 'qqdmr_positive_word_proportion']  = perc_positive2
        file.loc[i, 'qqdmr_negative_word_proportion']  = perc_negative2
    else:
    #If the section 2 data is not available, all the below values should be 0
        file.iloc[i, 20:33] = 0


# Calculating the values for Risk factors section                  

    if (topic3_data != 0):  
    
    #Calculating the positive score, negative score and Polarity score
        pos_score3, neg_score3, pol_score3 = positive_negative_pololarity_score(topic3_data)
    
        file.loc[i, 'rf_positive_score'] = pos_score3
        file.loc[i, 'rf_negative_score'] = neg_score3 
        file.loc[i, 'rf_polarity_score'] = pol_score3
    #Calculating the average sentence length for our section of data
        avg_sent_len_val3 = avg_sentence_len(topic3_data)
        file.loc[i, 'rf_average_sentence_length'] = avg_sent_len_val3
    #Calculating the word count, complex word count and percentage of complex words
        complex_count = 0
        word_count3, complex_count3, perc_complex_words3 = complex_word_count(topic3_data, complex_count)
        file.loc[i, 'rf_word_count']  = word_count3
        file.loc[i, 'rf_complex_word_count']  = complex_count3
        file.loc[i, 'rf_percentage_of_complex_words'] = perc_complex_words3
    #Calculating the Fog index value 
        fog_index3 = 0.4 * (avg_sent_len_val3 + perc_complex_words3)
        file.loc[i, 'rf_fog_index'] = fog_index3
    #Calculating the uncertainity score and constraining score
        uncer_score3, constr_score3 =uncertainty_constraining_score(topic3_data)
        perc_uncer3  = uncer_score3/word_count3
        perc_constr3 = constr_score3/word_count3
        file.loc[i, 'rf_uncertainty_score'] = uncer_score3
        file.loc[i, 'rf_constraining_score'] = constr_score3
    #Calculating the Uncertainity and Constraining word proportion
        file.loc[i, 'rf_uncertainty_word_proportion']  = perc_uncer3
        file.loc[i, 'rf_constraining_word_proportion'] = perc_constr3
    
    #Calculating the Positive and Negative word proportion
        perc_positive3 = pos_score3/word_count3
        perc_negative3 = neg_score3/word_count3
        file.loc[i, 'rf_positive_word_proportion']  = perc_positive3
        file.loc[i, 'rf_negative_word_proportion']  = perc_negative3

    else:
    #If the section 1 data is not available, all the below values should be 0
        file.iloc[i, 34:47] = 0

  #Clearing all the variables
    pos_score1 = neg_score1 = pol_score1 = avg_sent_len_val1 = word_count1 = complex_count1 = perc_complex_words1 = fog_index1 = uncer_score1 = constr_score1 = perc_uncer1 = perc_constr1 = perc_positive1 = perc_negative1 = 0
    pos_score2 = neg_score2 = pol_score2 = avg_sent_len_val2 = word_count2 = complex_count2 = perc_complex_words2 = fog_index2 = uncer_score2 = constr_score2 = perc_uncer2 = perc_constr2 = perc_positive2 = perc_negative2 = 0
    pos_score3 = neg_score3 = pol_score3 = avg_sent_len_val3 = word_count3 = complex_count3 = perc_complex_words3 = fog_index3 = uncer_score3 = constr_score3 = perc_uncer3 = perc_constr3 = perc_positive3 = perc_negative3 = 0
    constr_whole_count = 0

print("All operations on the dataframe have finished successfully")


All operations on the dataframe have finished successfully


In [129]:
#final output
#Removing the pre-fix http link at the beginning to keep values same as original file
link = 'https://www.sec.gov/Archives/'
secfname_or = secfname_or.reset_index(drop=True)
file['SECFNAME'] = secfname_or 

#Final output is stored in file dataframe
file

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,edgar/data/3662/0000950170-98-000413.txt,20,71,-0.56044,25,...,0,0,0,0,0,0,0,0,NaN,1452
1,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,edgar/data/3662/0000950170-98-001001.txt,11,54,-0.661538,28,...,0,0,0,0,0,0,0,0,NaN,1029
2,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,5
3,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,edgar/data/3662/0000950170-98-002145.txt,40,131,-0.532164,24,...,0,0,0,0,0,0,0,0,NaN,691
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0000012239,SPHERIX INC,200704,2007-04-02 00:00:00,10-K,edgar/data/12239/0001104659-07-024804.txt,109,124,-0.064378,17,...,6.597816,5887,11904,0,0,0.011257,0.015541,0.0,0.0,130
148,0000012239,SPHERIX INC,200705,2007-05-16 00:00:00,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,0
149,0000012239,SPHERIX INC,200705,2007-05-18 00:00:00,10-Q,edgar/data/12239/0001104659-07-041441.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,21
150,0000012239,SPHERIX INC,200705,2007-05-23 00:00:00,10-K/A,edgar/data/12239/0001104659-07-042333.txt,109,124,-0.064378,16,...,6.59721,5831,11827,0,0,0.01133,0.015642,0.0,0.0,129


In [ ]:
############### END OF THE CODE       ############